# Inspecting a RooWorkspace

What's inside a RooWorkspace created with HistFactory? Let's find out!

In [ ]:
!ls ../ws

The output folder of a HistFactory run usually contains many files, one of which is quite meaningful: it's the one with the name `combined`.

In [ ]:
f = new TFile("../ws/ICTPws_test_combined_ICTPws_test_model.root")

In [ ]:
f->ls()

The file contains:
- the `RooWorkspace` object, i.e. our likelihood model and associated datasets
- another object we shall neglect
- two directories containing the input histograms we used to build the workspace, one per channel (statistically independent set of data, cf. previous steps of this tutorial)
- the Measurement object representing what we created with create_workspace.ipynb, so effectively the specification of what is inside the Workspace

In case you ever panick and don't remember what's inside a workspace, you may do

In [ ]:
ICTPws_test->PrintXML("blabla")

and then the `blabla` folder (which is created just in case) contains the XML files which specify, in some convoluted version of the English language, what's inside the workspace:

In [ ]:
!ls blabla

In [ ]:
!cat blabla/ICTPws_test.xml

Now, back on the workspace, which is usually called `combined`. What's inside it? We check it with

In [ ]:
w = dynamic_cast<RooWorkspace*>(f->Get("combined"));
w->Print("")

The workspace is actually a collection of stuff:
- variables (which can be parameters, observables, constants... anything!)
- probability density functions (PDFs), i.e. mathematical functions of those variables (technically there is no distinction between observables and parameters at this stage)
- functions, which - differently from PDFs - aren't supposed to be normalized to 1 (for example, a histogram can be seen as a function of an observable, whose integral is equal to a given number of events)
- datasets, i.e. real data (the `Data` input histograms we gave when creating the workspace) and so-called Asimov data, which represent the overall _nominal_ background expectation;
- embedded datasets, i.e. internal representations of histograms
- parameter snapshots, i.e. "named copies" of the value (e.g. `9.888`) and settings (e.g. up and down error, `isConstant=kTRUE`) of a set of parameters
- named sets, i.e. "named sets" of parameters, which share the same meaning - e.g. "what's or what are the POI[s]", "which are the observables", etc.
- generic objects, like the notable HistFactory `ModelConfig` object, which represents how we should be putting together the overall likelihood model (PDF, data, and notion of who's the POI and who are the observable, the nuisance parameters and the constant parameters in our statistical task)

Please note that the meaning of _nominal_ depends strongly on the default parameters which were set in the creation of the workspace. More specifically: normalization parameters (such as the signal strength, which is usually the POI, and the background normalization factors) are taken at their nominal value as declared in the workspace creation. Typical mistake: expecting the Asimov dataset to represent the _background-only hypothesis_ when you set the signal strength default value to `1`.

We can for example identify what's the POI, and what are its default value and bounds:

In [ ]:
w->set("ModelConfig_POI")->Print("v")

or understand what are the observables,

In [ ]:
w->set("observables")->Print("v")

or we may do that via the ModelConfig object,

In [ ]:
mc = dynamic_cast<RooStats::ModelConfig*>(w->obj("ModelConfig"));
mc->GetParametersOfInterest()->Print();
mc->GetObservables()->Print();

and inspect the variable info via

In [ ]:
w->var("mu_ttH")->Print()

We can also perform some simple plotting, provided we choose the observables to look at. HistFactory workspaces always have two special ones:
- `channelCat` represents the category the events fall in (i.e. it's like an enum representing which of the various channels events belong to - for two channels, you'll see two _Possible states_ this discrete variable can take)
- `weightVar` represents instead the weight which may be associated to each event

In [ ]:
w->cat("channelCat")->Print("v")

Verbose printouts (`Print("v")`) and tree printouts (`Print("t")`) help nerds understand what's going on - how a variable or a function or in general any RooFit object is defined, which functions use it, and so on and so forth.